In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import pandas as pd
import lightgbm as lgbm

### Read toxic and non-toxic data

In [2]:
toxic_df = pd.read_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/Toxicity_prediction/toxic_predict_data/toxic_offtarget_profile.csv')
not_toxic_df = pd.read_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/Toxicity_prediction/toxic_predict_data/nontoxic_offtarget_profile.csv')
toxic_smiles = toxic_df['smiles'].tolist()
not_toxic_smiles = not_toxic_df['smiles'].tolist()

In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem

def smiles_to_fingerprint(smiles):
    mols = list(map(lambda x: Chem.MolFromSmiles(x), smiles))
    fingerprint = np.array(list(map(lambda x: AllChem.GetMorganFingerprintAsBitVect(x,2,1024), mols))) 
    
    return fingerprint

In [4]:
toxic_fingerprint = smiles_to_fingerprint(toxic_smiles)
not_toxic_fingerprint = smiles_to_fingerprint(not_toxic_smiles)

toxic_df = pd.DataFrame(toxic_fingerprint,index=toxic_smiles,columns=range(1024))
not_toxic_df = pd.DataFrame(not_toxic_fingerprint,index=not_toxic_smiles,columns=range(1024))
toxic_df.reset_index(inplace=True)
toxic_df.rename(columns={'index':'smiles'},inplace=True)
not_toxic_df.reset_index(inplace=True)
not_toxic_df.rename(columns={'index':'smiles'},inplace=True)

toxic_df['label'] = 1
not_toxic_df['label'] = 0

In [5]:
df = pd.concat([toxic_df, not_toxic_df], axis=0)
data_df = df.drop(['smiles','label'], axis=1)
print(data_df.shape)

(2106, 1024)


In [6]:
train_x, test_x, train_y, test_y = train_test_split(data_df, df['label'], test_size=0.2, random_state=999)

### Model hyperparameter selection

In [14]:
import optuna  
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [15]:
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    # Parameter grid
    param_grid = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 0.5, step=0.05), 
        "num_leaves": trial.suggest_int("num_leaves", 20, 1000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 500, step=100), 
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15, step=0.1),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0, step=0.1), 
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.9, step=0.1),
        "random_state": 2023,
    }

    X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1,random_state=999) 
    
    model = lgbm.LGBMClassifier(objective="binary", **param_grid) 
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="auc", 
        callbacks=[
            LightGBMPruningCallback(trial, "auc"),
            lgbm.early_stopping(20),
        ],
    )
    
    preds = model.predict_proba(X_val)
    auc = roc_auc_score(y_val, preds[:, 1])
    return auc


In [16]:

study = optuna.create_study(direction="maximize", study_name="LGBM Classifier") 
func = lambda trial: objective(trial, train_x, train_y)
study.optimize(func, n_trials=1000) 

[I 2023-09-10 06:31:22,273] A new study created in memory with name: LGBM Classifier
[I 2023-09-10 06:31:22,433] Trial 0 finished with value: 0.5 and parameters: {'n_estimators': 1000, 'learning_rate': 0.35, 'num_leaves': 200, 'max_depth': 12, 'min_data_in_leaf': 200, 'lambda_l1': 40, 'lambda_l2': 50, 'min_gain_to_split': 4.5, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 0.5.


[LightGBM] [Warning] lambda_l1 is set=40, reg_alpha=0.0 will be ignored. Current value: lambda_l1=40
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=4.5, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=4.5
[LightGBM] [Warning] lambda_l2 is set=50, reg_lambda=0.0 will be ignored. Current value: lambda_l2=50
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5	valid_0's binary_logloss: 0.6766
[LightGBM] [Warning] lambda_l1 is set=35, reg_al

[I 2023-09-10 06:31:22,587] Trial 1 finished with value: 0.602140361094933 and parameters: {'n_estimators': 900, 'learning_rate': 0.5, 'num_leaves': 400, 'max_depth': 5, 'min_data_in_leaf': 100, 'lambda_l1': 35, 'lambda_l2': 35, 'min_gain_to_split': 5.5, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 1 with value: 0.602140361094933.
[I 2023-09-10 06:31:22,690] Trial 2 finished with value: 0.5 and parameters: {'n_estimators': 500, 'learning_rate': 0.30000000000000004, 'num_leaves': 960, 'max_depth': 4, 'min_data_in_leaf': 500, 'lambda_l1': 55, 'lambda_l2': 15, 'min_gain_to_split': 2.3000000000000003, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. Best is trial 1 with value: 0.602140361094933.


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.60214	valid_0's binary_logloss: 0.66844
[LightGBM] [Warning] lambda_l1 is set=55, reg_alpha=0.0 will be ignored. Current value: lambda_l1=55
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.6000000000000001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6000000000000001
[LightGBM] [Warning] min_data_in_leaf is set=500, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=500
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=2.3000000000000003, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.3000000000000003
[LightGBM] [Warning] lambda_l2 is set=15, reg_lambda=0.0 will be ignored. Current val

[I 2023-09-10 06:31:22,819] Trial 3 finished with value: 0.5 and parameters: {'n_estimators': 500, 'learning_rate': 0.25, 'num_leaves': 120, 'max_depth': 6, 'min_data_in_leaf': 400, 'lambda_l1': 85, 'lambda_l2': 90, 'min_gain_to_split': 10.600000000000001, 'bagging_fraction': 0.30000000000000004, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 1 with value: 0.602140361094933.
[I 2023-09-10 06:31:22,928] Trial 4 finished with value: 0.5 and parameters: {'n_estimators': 800, 'learning_rate': 0.35, 'num_leaves': 920, 'max_depth': 8, 'min_data_in_leaf': 400, 'lambda_l1': 95, 'lambda_l2': 95, 'min_gain_to_split': 3.2, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 1 with value: 0.602140361094933.


...

Training until validation scores don't improve for 20 rounds


[I 2023-09-10 06:37:09,614] Trial 998 pruned. Trial was pruned at iteration 29.


Training until validation scores don't improve for 20 rounds


[I 2023-09-10 06:37:10,035] Trial 999 pruned. Trial was pruned at iteration 25.


Training until validation scores don't improve for 20 rounds


In [17]:
# print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest value (auc): {study.best_value:.5f}")
print(f"\tBest params:")
for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (auc): 0.86488
	Best params:
		n_estimators: 900
		learning_rate: 0.45000000000000007
		num_leaves: 940
		max_depth: 9
		min_data_in_leaf: 100
		lambda_l1: 0
		lambda_l2: 10
		min_gain_to_split: 0.0
		bagging_fraction: 1.0
		bagging_freq: 1
		feature_fraction: 0.5


### Conduct five-fold cross-training based on the optimum hyperparameter

In [7]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, balanced_accuracy_score, matthews_corrcoef
import lightgbm as lgbm

# 创建 KFold 实例
kf = KFold(n_splits=5, random_state=999, shuffle=True)

train_x, test_x, train_y, test_y = train_test_split(data_df, df['label'], test_size=0.2, random_state=999)

# model = lgbm.LGBMClassifier(**study.best_params)
model = lgbm.LGBMClassifier(n_estimators=900,
                            learning_rate=0.45,
                            num_leaves=940,
                            max_depth=9,
                            min_data_in_leaf=100,
                            lambda_l1=0,
                            lambda_l2=10,
                            min_gain_to_split=0.0,
                            bagging_fraction=0.7,
                            bagging_freq=1,
                            feature_fraction=0.5,
                            random_state=2023)

acc_list = []
auc_list = []
f1_list = []
bacc_list = []
mcc_list = []

for train_index,val_index in kf.split(train_x):
    X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(test_x)
    y_prob = model.predict_proba(test_x)

    acc_list.append(accuracy_score(test_y, y_pred))
    auc_list.append(roc_auc_score(test_y, y_prob[:,1]))
    f1_list.append(f1_score(test_y, y_pred, average='binary'))
    bacc_list.append(balanced_accuracy_score(test_y, y_pred))
    mcc_list.append(matthews_corrcoef(test_y, y_pred))

print('accuracy_score:', np.mean(acc_list), np.std(acc_list))
print('roc_auc_score:', np.mean(auc_list), np.std(auc_list))
print('f1_score:', np.mean(f1_list), np.std(f1_list))
print('balanced_accuracy_score:', np.mean(bacc_list), np.std(bacc_list))
print('matthews_corrcoef:', np.mean(mcc_list), np.std(mcc_list))

[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Warning] lambda_l2 is set=10, reg_lambda=0.0 will be ignored. Current value: lambda_l2=10
[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging

#### Save model performance results

In [9]:
result_df_ecfp = pd.DataFrame({'accuracy_score':acc_list,
                                'roc_auc_score':auc_list,
                                'f1_score':f1_list,
                                'balanced_accuracy_score':bacc_list,
                                'matthews_corrcoef':mcc_list})
# result_df_ecfp.head(5)
result_df_ecfp.to_csv('/home/liujin/Offtarget_drugsafety/offtarget_application/Toxicity_prediction/picture_draw/result_df_ecfp.csv',index=False)